<a href="https://colab.research.google.com/github/ezinneanne/DeepTechReady-Work/blob/deepdev/Week1_assign2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

assignment 2

I've prepared an assignment for you The data set for this assignment is a house pricing data set which is similar to this session demo data.

The goal of this assignment is to build and compare two models on classification and regression.

You are to choose a classification and regression model that we have explored during the demo and another classification and regression model not explored in our demo.

An example for classification is using random forest classifier and an example of a regression model is support vector machine.

Do not fret as you can find these models and more about their documentation on the Sci-learn website.

There are also relevant links that I've provided in the Jupiter notebook to see more of these models. Finally you are required to evaluate the models in each category to determine which performs best for predicting the house price and the house price category.

Remember to upload your completed assignment to the learning portal.

The full assignment details and the deliverables you are required to submit will also be made available on the learning portal. If you do encounter any challenges don't hesitate to reach out to a teaching assistant for supports You can also engage via Google Classroom for additional guidance and clarification.

In [ ]:
#model used in the demo
#linear regression for regression
#logistic regression for classification

#models not used in the demo
#randomforestclassifier for classification
#randomforestregressor for regression

In [1]:
# Mounting gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

In [3]:
# Set parameters
DATA_DIR = '/content/drive/MyDrive/'


# Reading the dataset
data = pd.read_csv(DATA_DIR + 'nigeria_houses_cc.csv', encoding='latin1')
# display the first 5 rows
data.head()

,bedrooms,bathrooms,toilets,parking_space,title,town,state,price,price Category
0,6.0,5.0,5.0,4.0,Detached Duplex,Mabushi,Abuja,450000000,1
1,4.0,5.0,5.0,4.0,Terraced Duplexes,Katampe,Abuja,800000000,1
2,4.0,5.0,5.0,4.0,Detached Duplex,Lekki,Lagos,120000000,1
3,4.0,4.0,5.0,6.0,Detached Duplex,Ajah,Lagos,40000000,1
4,4.0,4.0,5.0,2.0,Semi Detached Duplex,Lekki,Lagos,75000000,1


In [4]:
# Basic cleanup: Drop duplicates and rows with missing values
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13888 entries, 0 to 24324
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bedrooms        13888 non-null  float64
 1   bathrooms       13888 non-null  float64
 2   toilets         13888 non-null  float64
 3   parking_space   13888 non-null  float64
 4   title           13888 non-null  object 
 5   town            13888 non-null  object 
 6   state           13888 non-null  object 
 7   price           13888 non-null  int64  
 8   price Category  13888 non-null  int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 1.1+ MB


In [5]:
data.reset_index(drop=True, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13888 entries, 0 to 13887
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bedrooms        13888 non-null  float64
 1   bathrooms       13888 non-null  float64
 2   toilets         13888 non-null  float64
 3   parking_space   13888 non-null  float64
 4   title           13888 non-null  object 
 5   town            13888 non-null  object 
 6   state           13888 non-null  object 
 7   price           13888 non-null  int64  
 8   price Category  13888 non-null  int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 976.6+ KB


In [6]:
# Apply log transformation to price for regression stability
data['log_price'] = np.log1p(data['price'])

# Define features and target for regression
X_reg = data.drop(columns=['price', 'log_price', 'price Category'])
y_reg = data['log_price']

# Define features and target for classification
X_clf = data.drop(columns=['price', 'log_price', 'price Category'])
y_clf = data['price Category']

# Identify categorical and numerical features
categorical_features = ['title', 'town', 'state']
numerical_features = ['bedrooms', 'bathrooms', 'toilets', 'parking_space']

# Create preprocessing pipelines
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('num', StandardScaler(), numerical_features)
])

In [7]:
# ----------------- Regression Models ------------------
# Linear Regression pipeline
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Random Forest Regressor pipeline
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [8]:
# Train-test split for regression
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Train models
lr_pipeline.fit(X_reg_train, y_reg_train)
rf_pipeline.fit(X_reg_train, y_reg_train)

# Predict and evaluate
lr_preds = lr_pipeline.predict(X_reg_test)
rf_preds = rf_pipeline.predict(X_reg_test)
lr_mse = mean_squared_error(y_reg_test, lr_preds)
lr_r2 = r2_score(y_reg_test, lr_preds)
rf_mse = mean_squared_error(y_reg_test, rf_preds)
rf_r2 = r2_score(y_reg_test, rf_preds)

In [9]:
# Evaluation
print("Linear Regression MSE:", lr_mse)
print("Linear Regression R2:", lr_r2)

print("Random Forest Regressor MSE:", rf_mse)
print("Random Forest Regressor R2:", rf_r2)

Linear Regression MSE: 0.5641295545519075
Linear Regression R2: 0.6111562823059982
Random Forest Regressor MSE: 0.6029511511906918
Random Forest Regressor R2: 0.5843972978811662


In [10]:
# ----------------- Classification Models ------------------
# Calculate class weights for logistic regression
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_clf), y=y_clf)
class_weight_dict = dict(zip(np.unique(y_clf), class_weights))

# Logistic Regression pipeline
log_clf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, class_weight=class_weight_dict))
])

# Random Forest Classifier pipeline
rf_clf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

In [11]:
# Train-test split for classification
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

# Train models
log_clf_pipeline.fit(X_clf_train, y_clf_train)
rf_clf_pipeline.fit(X_clf_train, y_clf_train)

# Predict and evaluate
log_preds = log_clf_pipeline.predict(X_clf_test)
rf_preds = rf_clf_pipeline.predict(X_clf_test)
log_report = classification_report(y_clf_test, log_preds, output_dict=True)
rf_report = classification_report(y_clf_test, rf_preds, output_dict=True)

In [12]:
# Evaluation
print("Logistic Regression Report:\n", log_report)
print("Random Forest Classifier Report:\n", rf_report)

Logistic Regression Report:
 {'0': {'precision': 0.05263157894736842, 'recall': 0.6333333333333333, 'f1-score': 0.09718670076726342, 'support': 30.0}, '1': {'precision': 0.9954489035995036, 'recall': 0.8755458515283843, 'f1-score': 0.9316553727008713, 'support': 2748.0}, 'accuracy': 0.8729301655867531, 'macro avg': {'precision': 0.524040241273436, 'recall': 0.7544395924308588, 'f1-score': 0.5144210367340674, 'support': 2778.0}, 'weighted avg': {'precision': 0.9852672910222668, 'recall': 0.8729301655867531, 'f1-score': 0.9226438319672471, 'support': 2778.0}}
Random Forest Classifier Report:
 {'0': {'precision': 0.015384615384615385, 'recall': 0.03333333333333333, 'f1-score': 0.021052631578947368, 'support': 30.0}, '1': {'precision': 0.9893107261334316, 'recall': 0.9767103347889374, 'f1-score': 0.9829701519868156, 'support': 2748.0}, 'accuracy': 0.9665226781857451, 'macro avg': {'precision': 0.5023476707590235, 'recall': 0.5050218340611354, 'f1-score': 0.5020113917828815, 'support': 2778

In [13]:
import joblib

# Save models and pipeline
joblib.dump(rf_pipeline, 'rf_regressor.pkl')
joblib.dump(rf_clf_pipeline, 'rf_classifier.pkl')
joblib.dump(preprocessor, 'preprocessor.pkl')


['preprocessor.pkl']

In [14]:
#downloading the files

from google.colab import files
files.download('rf_regressor.pkl')
files.download('rf_classifier.pkl')
files.download('preprocessor.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ------------------ Feature Engineering ------------------

# Add total rooms
data['total_rooms'] = data['bedrooms'] + data['bathrooms'] + data['toilets']

# Add room to parking ratio
data['room_parking_ratio'] = data['total_rooms'] / (data['parking_space'] + 1)

# Add binary flag for Abuja and Lagos
data['is_abuja'] = (data['state'].str.lower() == 'abuja').astype(int)
data['is_lagos'] = (data['state'].str.lower() == 'lagos').astype(int)

# Add binary luxury feature (simplified logic for now)
data['is_luxury'] = data['title'].str.contains('Detached|Terraced', case=False).astype(int)

# Features and target variables
X = data.drop(columns=['price', 'price Category'])
y_reg = data['price']
y_clf = data['price Category']

# Split data
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X, y_reg, test_size=0.2, random_state=42)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X, y_clf, test_size=0.2, random_state=42)

# ------------------ Preprocessing ------------------

numeric_features = ['bedrooms', 'bathrooms', 'toilets', 'parking_space', 'total_rooms', 'room_parking_ratio']
categorical_features = ['title', 'town', 'state']
binary_features = ['is_abuja', 'is_lagos', 'is_luxury']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('bin', 'passthrough', binary_features)
])

In [ ]:
# ------------------ Classification Models ------------------

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_clf_train), y=y_clf_train)
class_weight_dict = dict(zip(np.unique(y_clf_train), class_weights))

log_clf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, class_weight=class_weight_dict))
])

rf_clf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# Hyperparameter tuning for logistic regression
param_grid_logreg = {
    'classifier__C': [0.01, 0.1, 1],
    'classifier__solver': ['liblinear']
}

log_grid_search = GridSearchCV(log_clf_pipeline, param_grid_logreg, cv=5, scoring='f1_weighted')
log_grid_search.fit(X_clf_train, y_clf_train)
log_best_model = log_grid_search.best_estimator_

# Hyperparameter tuning for random forest classifier
param_grid_rf_clf = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [10, None],
    'classifier__min_samples_split': [2, 5]
}

rf_grid_search = GridSearchCV(rf_clf_pipeline, param_grid_rf_clf, cv=5, scoring='f1_weighted')
rf_grid_search.fit(X_clf_train, y_clf_train)
rf_best_model = rf_grid_search.best_estimator_

In [ ]:
# ------------------ Regression Models ------------------

lin_reg_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

rf_reg_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

lin_reg_pipeline.fit(X_reg_train, y_reg_train)
rf_reg_pipeline.fit(X_reg_train, y_reg_train)

# Predictions
y_lin_pred = lin_reg_pipeline.predict(X_reg_test)
y_rf_pred = rf_reg_pipeline.predict(X_reg_test)

# Evaluation
lin_mse = mean_squared_error(y_reg_test, y_lin_pred)
lin_r2 = r2_score(y_reg_test, y_lin_pred)

rf_mse = mean_squared_error(y_reg_test, y_rf_pred)
rf_r2 = r2_score(y_reg_test, y_rf_pred)

# Classification reports
y_log_pred = log_best_model.predict(X_clf_test)
y_rf_clf_pred = rf_best_model.predict(X_clf_test)

log_report = classification_report(y_clf_test, y_log_pred, output_dict=True)
rf_clf_report = classification_report(y_clf_test, y_rf_clf_pred, output_dict=True)

(lin_mse, lin_r2, rf_mse, rf_r2, log_grid_search.best_params_, rf_grid_search.best_params_, log_report, rf_clf_report)

(1.0419546746198868e+19,
 -0.11186341495839813,
 2.3976233614685802e+20,
 -24.58489119921656,
 {'classifier__C': 1, 'classifier__solver': 'liblinear'},
 {'classifier__max_depth': None,
  'classifier__min_samples_split': 2,
  'classifier__n_estimators': 100},
 {'0': {'precision': 0.04207920792079208,
   'recall': 0.5666666666666667,
   'f1-score': 0.07834101382488479,
   'support': 30.0},
  '1': {'precision': 0.9945240101095199,
   'recall': 0.8591703056768559,
   'f1-score': 0.9219055056618508,
   'support': 2748.0},
  'accuracy': 0.8560115190784737,
  'macro avg': {'precision': 0.518301609015156,
   'recall': 0.7129184861717612,
   'f1-score': 0.5001232597433678,
   'support': 2778.0},
  'weighted avg': {'precision': 0.984238429092363,
   'recall': 0.8560115190784737,
   'f1-score': 0.91279573793143,
   'support': 2778.0}},
 {'0': {'precision': 0.016129032258064516,
   'recall': 0.03333333333333333,
   'f1-score': 0.021739130434782608,
   'support': 30.0},
  '1': {'precision': 0.98932